## Data Exploration: Feature Engineering for Multinomial Epilepsy Classification

### In this notebook, a variety of feature extraction approaches are explored, including:

1. Fourier Transform
2. Discrete Wavelet Transform
3. Shannon Entropy
4. Primary Component Analysis (PCA)



### Setup

In [1]:
import numpy as np
import pandas as pd
import util

### Data
We first parse and normalize our dataset.

In [2]:
X, y, ids, chunks = util.parse_data()
n = X.shape[0]
classes = np.unique(y)
num_chunks = np.unique(chunks).shape[0]

# Set up evaluation metrics
scoring = util.get_scoring_metrics(classes)

### Discrete Wavelet Transform